In [1]:
import pyodbc

In [2]:
pyodbc.drivers()

['SQL Server',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']

In [3]:
from sqlalchemy import create_engine
import urllib

server = 'DESKTOP-6MJU11J'
database = 'iotdata_events' 

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Example test

with engine.connect() as conn:
    result = conn.execute("SELECT GETDATE()")
    for row in result:
        print(row)


(datetime.datetime(2025, 4, 24, 8, 57, 14, 723000),)


In [4]:
# Load data
import pandas as pd
table = "FireEvents" 
df = pd.read_sql(f"SELECT * FROM {table}", engine)
print(df.shape)
df.head()

(940083, 12)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,WILLSHOWINAPP,MESSAGE
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""initialising rl03 back of ..."
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 smoke zone 2 r..."
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing rl06 smoke ..."
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 back of house ..."
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing node 1 ffcp..."


In [5]:
import json

In [6]:
# Step 2: Load data from the SQL Server into a DataFrame
#table = "FireEvents"  # Name of your SQL table
##df = pd.read_sql(f"SELECT * FROM {table}", engine)

# Display the shape (rows, columns) of the loaded DataFrame
print("Original DataFrame shape:", df.shape)

# Show the first few rows to preview the structure
##df.head()

Original DataFrame shape: (940083, 12)


In [7]:
# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from EventDetails column:")
print(df['IDENTIFIER'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from SITE column:")
print(df['STATE'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from MESSAGE column:")
print(df['MESSAGE'].iloc[0])

# Step 3: Check structure of one JSON entry to verify format
print("Sample JSON from DURATION column:")
print(df['DURATION'].iloc[0])

Sample JSON from EventDetails column:
{
  "device": "172",
  "group": null,
  "loop": "26",
  "module": null,
  "node": "9",
  "sensor": "0",
  "zone": "109"
}
Sample JSON from SITE column:
{
  "alarm": "off",
  "fault": "off",
  "faultCode": null,
  "isolate": "off",
  "preAlarm": "off",
  "sensor": null
}
Sample JSON from MESSAGE column:
{
  "descriptor": "initialising rl03 back of house rl03 z1 corridor mcp 3012a",
  "detector": "Z109N9L26D172S0",
  "notification": "z109n9l26d172s0 - m214 - SC1 retail",
  "raw": "initialising rl03 back of house rl03 z1 corridor mcp 3012a",
  "state": "NNNN"
}
Sample JSON from DURATION column:
None


In [8]:
df=df.join(df['IDENTIFIER'].apply(json.loads).apply(pd.Series))

In [9]:
print(df.shape)
df.head()

(940083, 19)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,WILLSHOWINAPP,MESSAGE,device,group,loop,module,node,sensor,zone
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""initialising rl03 back of ...",172,None,26,None,9,0,109
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 smoke zone 2 r...",191,None,26,None,9,0,102
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing rl06 smoke ...",92,None,13,None,5,0,191
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,True,"{\n ""descriptor"": ""normal rl03 back of house ...",172,None,26,None,9,0,109
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,True,"{\n ""descriptor"": ""device missing node 1 ffcp...",136,None,3,None,1,0,801


In [10]:
df = df.join(df['STATE'].apply(json.loads).apply(pd.Series), rsuffix='_from_state')

In [11]:
print(df.shape)
df.head()

(940083, 25)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,module,node,sensor,zone,alarm,fault,faultCode,isolate,preAlarm,sensor_from_state
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,109,off,off,None,off,off,None
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,102,off,off,None,off,off,None
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,5,0,191,unknown,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,9,0,109,off,off,None,off,off,None
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,1,0,801,unknown,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None


In [12]:
df=df.join(df['MESSAGE'].apply(json.loads).apply(pd.Series))


In [13]:
print(df.shape)
df.head()

(940083, 30)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,fault,faultCode,isolate,preAlarm,sensor_from_state,descriptor,detector,notification,raw,state
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,None,xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,None,EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,None,4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,off,None,off,off,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,None,91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,on,"{'panel': 'AXIS', 'text': None, 'value': None}",unknown,unknown,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?


In [14]:
print(df.shape)

(940083, 30)


In [15]:
df['DURATION'] = df['DURATION'].fillna('{}')


In [16]:
print(df['DURATION'].dtype)
print(df['DURATION'].head())

object
0    {}
1    {}
2    {}
3    {}
4    {}
Name: DURATION, dtype: object


In [17]:
df_duration_expanded = df['DURATION'].apply(json.loads).apply(pd.Series, dtype='object')


In [18]:

#Rename columns with a prefix
df_duration_expanded = df_duration_expanded.rename(columns=lambda x: f'duration_{x}')

In [19]:
df = df.join(df_duration_expanded)

In [20]:
print (df.shape)
df.head()

(940083, 34)


,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaN,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaN,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaN,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaN,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaN,NaN


In [21]:
df.loc[2, 'state']

'??F?'

In [22]:
df.loc[2, 'state']

'??F?'

In [23]:
df['duration_isolate']

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
            ...   
940078         NaN
940079         NaN
940080         NaN
940081    PT0.328S
940082        None
Name: duration_isolate, Length: 940083, dtype: object

In [41]:
!pip install isodate

In [24]:
from isodate import parse_duration
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT
df['duration_isolate'] = df['duration_isolate'].apply(safe_parse_duration)



In [25]:
df['duration_isolate']


0                           NaT
1                           NaT
2                           NaT
3                           NaT
4                           NaT
                  ...          
940078                      NaT
940079                      NaT
940080                      NaT
940081   0 days 00:00:00.328000
940082                      NaT
Name: duration_isolate, Length: 940083, dtype: timedelta64[ns]

In [26]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,IDENTIFIER,RECORDEDAT,SITEID,STATE,SYSTEMGENERATED,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,"{\n ""device"": ""191"",\n ""group"": null,\n ""lo...",2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,"{\n ""device"": ""92"",\n ""group"": null,\n ""loo...",2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,"{\n ""device"": ""172"",\n ""group"": null,\n ""lo...",2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""off"",\n ""fault"": ""off"",\n ""fa...",False,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,"{\n ""device"": ""136"",\n ""group"": null,\n ""lo...",2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,"{\n ""alarm"": ""unknown"",\n ""fault"": ""on"",\n ...",False,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [27]:
del df['STATE']
del df['IDENTIFIER']
del df['MESSAGE']


In [28]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaN,NaN,NaT,NaN
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaN,NaN,NaT,NaN
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaN,NaN,NaT,NaN
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaN,NaN,NaT,NaN
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaN,NaN,NaT,NaN


In [30]:
df['faultCode']


0                                                      None
1                                                      None
2            {'panel': 'AXIS', 'text': None, 'value': None}
3                                                      None
4            {'panel': 'AXIS', 'text': None, 'value': None}
                                ...                        
940078    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940079    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940080    {'panel': 'SIMPLEX', 'text': None, 'value': None}
940081                                                 None
940082                                                 None
Name: faultCode, Length: 940083, dtype: object

In [63]:
# Step 2: Convert JSON string to Python dict
df_json = df['DURATION'].fillna('{}').apply(json.loads)

# Step 3: Extract 'isolate' using safe .get()
df['duration_isolate'] = df_json.apply(lambda x: x.get('isolate'))

In [64]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_isolate'] = df['duration_isolate'].apply(safe_parse_duration)

In [33]:
print(df['duration_isolate'].notna().sum(), "parsed durations")
print(df['duration_isolate'].unique())



116795 parsed durations
[          'NaT' 174968573000000 604938119000000 ... 120254947000000
  22186931000000   9624483000000]


In [34]:
print(df['duration_isolate'].head(20))


0                       NaT
1                       NaT
2                       NaT
3                       NaT
4                       NaT
5                       NaT
6                       NaT
7                       NaT
8                       NaT
9                       NaT
10                      NaT
11                      NaT
12                      NaT
13                      NaT
14                      NaT
15   2 days 00:36:08.573000
16                      NaT
17                      NaT
18                      NaT
19                      NaT
Name: duration_isolate, dtype: timedelta64[ns]


In [35]:
# Step 2: Convert JSON string to Python dict
df_json = df['DURATION'].fillna('{}').apply(json.loads)

# Step 3: Extract 'isolate' using safe .get()
df['duration_alarm'] = df_json.apply(lambda x: x.get('alarm'))

In [36]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_alarm'] = df['duration_alarm'].apply(safe_parse_duration)

In [37]:
print(df['duration_alarm'].notna().sum(), "parsed durations")
print(df['duration_alarm'].unique())



3062 parsed durations
[       'NaT' 299983000000 534424000000 ... 146212000000  50137000000
 188848000000]


In [40]:
df[df['duration_alarm'].notna()].head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
208,2025-01-18 02:03:29.814,n7Tktl6xJA2qDV8RcMpf,KgoXC0AcdzVFkdSij5X3,"{\n ""alarm"": ""PT4M59.983S"",\n ""fault"": null,...",77JEVYiHkRp5Ly2mVuc3,2025-01-18 02:03:14.104,DO05xnc9uzEwZ1OvRGSP,False,True,42,...,None,normal rl01 back of house rl01 electrical rm r...,Z49N1L2D42S0,z49n1l2d42s0 - SC1 retail - m214,normal rl01 back of house rl01 electrical rm r...,NNNN,0 days 00:04:59.983000,None,NaT,None
386,2024-12-11 12:59:09.451,n7Tktl6xJA2qDV8RcMpf,aYonkgIQXXmwR7vl38JI,"{\n ""alarm"": ""PT8M54.424S"",\n ""fault"": null,...",Ie0f29pgNbkKT3Uk8vQy,2024-12-11 12:59:07.225,DO05xnc9uzEwZ1OvRGSP,False,True,118,...,None,normal sprinkler electric pump b01 z1 electric...,Z405N1L3D118S0,z405n1l3d118s0 - SC1 retail - m214,normal sprinkler electric pump b01 z1 electric...,NNNN,0 days 00:08:54.424000,None,NaT,None
822,2025-01-22 04:03:18.843,n7Tktl6xJA2qDV8RcMpf,BDRlKbJKUui43SWY3Oez,"{\n ""alarm"": ""PT33M52.3S"",\n ""fault"": null,\...",ct7OPlGLtwxkTEeMJQES,2025-01-22 04:03:18.173,DO05xnc9uzEwZ1OvRGSP,False,True,42,...,None,reset rl03 smoke zone 1 rl03 sz1 mall adj s3027,Z101N9L26D42S0,z101n9l26d42s0 - SC1 retail - m214,reset rl03 smoke zone 1 rl03 sz1 mall adj s3027,NNNN,0 days 00:33:52.300000,None,NaT,None
962,2024-08-15 18:47:44.966,n7Tktl6xJA2qDV8RcMpf,aYonkgIQXXmwR7vl38JI,"{\n ""alarm"": ""PT5M56.371S"",\n ""fault"": null,...",ujNACLSpFmtQa3bHYNyP,2024-08-15 18:47:44.009,DO05xnc9uzEwZ1OvRGSP,False,True,118,...,None,normal sprinkler electric pump b01 z1 electric...,Z405N1L3D118S0,z405n1l3d118s0 - m214 - SC1 retail,normal sprinkler electric pump b01 z1 electric...,NNNN,0 days 00:05:56.371000,None,NaT,None
1047,2025-01-22 21:09:01.019,n7Tktl6xJA2qDV8RcMpf,VjnAcQNarNM8F0qlt9SE,"{\n ""alarm"": ""PT37M9.309S"",\n ""fault"": null,...",UocPiY76Jq1oggseIVAB,2025-01-22 21:08:59.895,DO05xnc9uzEwZ1OvRGSP,False,False,117,...,None,reset rl03 smoke zone 4 rl03 sz4 mall adj s3035,Z104N9L26D117S0,z104n9l26d117s0 - SC1 retail - m214,reset rl03 smoke zone 4 rl03 sz4 mall adj s3035,NNNN,0 days 00:37:09.309000,None,NaT,None


In [41]:
# Step 2: Convert JSON string to Python dict
df_json = df['DURATION'].fillna('{}').apply(json.loads)

# Step 3: Extract 'isolate' using safe .get()
df['duration_preAlarm'] = df_json.apply(lambda x: x.get('preAlarm'))

In [42]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_preAlarm'] = df['duration_preAlarm'].apply(safe_parse_duration)

In [45]:
df[df['duration_preAlarm'].notna()].head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
4933,2025-01-15 01:31:28.534,n7Tktl6xJA2qDV8RcMpf,voMInBJtPE3z26BFhBSb,"{\n ""alarm"": null,\n ""fault"": null,\n ""isol...",UvqRTauMTZYbZCZxkead,2025-01-15 01:31:27.439,DO05xnc9uzEwZ1OvRGSP,False,True,147,...,None,normal rl02 smoke zone 1 rl02 sz1 s2026a swaro...,Z71N6L6D147S0,z71n6l6d147s0 - SC1 retail - m214,normal rl02 smoke zone 1 rl02 sz1 s2026a swaro...,NNNN,NaT,None,NaT,0 days 00:00:33.864000
5530,2025-01-14 22:46:15.887,n7Tktl6xJA2qDV8RcMpf,i3mnEx3qA3Ib61cTCvFP,"{\n ""alarm"": null,\n ""fault"": null,\n ""isol...",PUhyERXMKp0hneCZrHCY,2025-01-14 22:46:15.081,DO05xnc9uzEwZ1OvRGSP,False,True,4,...,None,normal node 4 ffcp modules rl08 ahu 6001 duct ...,Z804N4L16D4S0,z804n4l16d4s0 - SC1 retail - m214,normal node 4 ffcp modules rl08 ahu 6001 duct ...,NNNN,NaT,None,NaT,0 days 00:12:48.740000
8670,2025-01-23 23:20:01.998,n7Tktl6xJA2qDV8RcMpf,7VteXSDOnOyBkenECFmL,"{\n ""alarm"": ""PT0.888S"",\n ""fault"": null,\n ...",WVxrazA8HXltQAyWF3Y7,2025-01-23 23:20:00.347,DO05xnc9uzEwZ1OvRGSP,False,True,125,...,None,normal rl03 smoke zone 4,Z104N9L25D125S0,z104n9l25d125s0 - SC1 retail - m214,normal rl03 smoke zone 4,NNNN,0 days 00:00:00.888000,None,NaT,0 days 00:00:25.066000
10153,2025-01-23 21:54:17.961,n7Tktl6xJA2qDV8RcMpf,P4RsvKFfiS7LFObhgr4D,"{\n ""alarm"": ""PT23M10.801S"",\n ""fault"": null...",In9JSfJbs3wwO1xQ6iec,2025-01-23 21:54:16.946,DO05xnc9uzEwZ1OvRGSP,False,True,11,...,None,reset rl02 back of house rl02 z2 gucci boh sto...,Z79N6L5D11S0,z79n6l5d11s0 - SC1 retail - m214,reset rl02 back of house rl02 z2 gucci boh sto...,NNNN,0 days 00:23:10.801000,None,NaT,0 days 00:23:31.090000
11660,2025-01-17 23:52:04.023,n7Tktl6xJA2qDV8RcMpf,H3X3mbBsA8sz8QBAPfZ5,"{\n ""alarm"": ""PT9.373S"",\n ""fault"": null,\n ...",CGnr6KKwZjot37QY83dt,2025-01-17 23:52:03.204,DO05xnc9uzEwZ1OvRGSP,False,True,154,...,None,normal rl06 smoke zone 1,Z194N5L13D154S0,z194n5l13d154s0 - SC1 retail - m214,normal rl06 smoke zone 1,NNNN,0 days 00:00:09.373000,None,NaT,0 days 00:00:27.948000


In [46]:
# Timestamp conversion
df['ADDEDAT'] = pd.to_datetime(df['ADDEDAT'], errors='coerce')


In [47]:
df['RECORDEDAT'] = pd.to_datetime(df['RECORDEDAT'], errors='coerce')

In [48]:
df.head(10)

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaT,NaN,NaT,NaT
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaT,NaN,NaT,NaT
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaT,NaN,NaT,NaT
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaT,NaN,NaT,NaT
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaT,NaN,NaT,NaT
5,2024-07-23 20:41:19.217,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},i3nf8E4FxlMW1ZzOy1h0,2024-07-23 20:41:17.453,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaT,NaN,NaT,NaT
6,2024-12-11 02:38:35.703,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},TnXI50CNHrOJnPRkUWlu,2024-12-11 02:38:34.384,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaT,NaN,NaT,NaT
7,2024-11-06 16:15:09.196,n7Tktl6xJA2qDV8RcMpf,C29g2RaN96YNvHFaLpiE,{},tWM6FMeNqljcdcavheuO,2024-11-06 16:15:08.310,DO05xnc9uzEwZ1OvRGSP,False,True,None,...,None,mute,Z923N23,z923n23 - SC1 retail - m214,mute,???I,NaT,NaN,NaT,NaT
8,2024-09-10 22:24:11.867,n7Tktl6xJA2qDV8RcMpf,8hQnV3aU7CaTupG0w389,{},gcEyBbBoj5dh8kirqCSy,2024-09-10 22:24:11.867,DO05xnc9uzEwZ1OvRGSP,True,True,195,...,"{'maximum': None, 'minimum': None, 'normal': N...",active event expiry,Z44N1L2D195S0,detector id: z44n1l2d195s0 - m214 - SC1 retail,???nz44n1l2d195s0 active event expiry,???N,NaT,NaN,NaT,NaT
9,2024-08-11 17:10:44.711,n7Tktl6xJA2qDV8RcMpf,nSHXqKSwHbGZjNi6GYEJ,{},IAB0C9HOKcT7zcBOOw9o,2024-08-11 17:10:34.232,DO05xnc9uzEwZ1OvRGSP,False,True,30,...,None,normal rl04 smoke zone 3,Z133N9L23D30S0,z133n9l23d30s0 - m214 - SC1 retail,normal rl04 smoke zone 3,NNNN,NaT,NaN,NaT,NaT


In [49]:
# Check for missing timestamps (NaT)
print(df['ADDEDAT'].isna().sum(), "missing timestamps")

# Check the first few rows to verify timestamp format
print(df['ADDEDAT'].head())

# Check for invalid timestamps (NaT or outliers)
print(df['ADDEDAT'].describe())


0 missing timestamps
0   2024-08-07 17:56:29.842
1   2024-08-14 10:49:42.359
2   2024-09-03 17:05:31.186
3   2025-01-09 03:27:26.923
4   2024-08-28 12:18:26.508
Name: ADDEDAT, dtype: datetime64[ns]
count                         940083
unique                        938202
top       2024-07-31 06:44:16.954000
freq                             421
first     2024-07-10 01:07:06.668000
last      2025-03-10 19:41:54.640000
Name: ADDEDAT, dtype: object


C:\Users\User\AppData\Local\Temp\ipykernel_11800\685933736.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df['ADDEDAT'].describe())


In [51]:
# ISO conversion of Duration_fault
#: Extract 'fault' using safe .get()
df['duration_fault'] = df_json.apply(lambda x: x.get('fault'))

In [52]:
# Step 4: Safe parse
def safe_parse_duration(x):
    try:
        if pd.isnull(x):
            return pd.NaT
        return parse_duration(x)
    except:
        return pd.NaT

# Step 5: Apply parsing
df['duration_fault'] = df['duration_fault'].apply(safe_parse_duration)

In [53]:
print(df['duration_fault'].notna().sum(), "parsed durations")
print(df['duration_fault'].unique())

174226 parsed durations
[          'NaT' 604860610000000   1726313000000 ... 120252943000000
  22184715000000   9627810000000]


In [56]:
# verify all the ISO are converted successfully
cols = ['duration_preAlarm', 'duration_fault', 'duration_alarm', 'duration_isolate']
df[cols][df[cols].notna().any(axis=1)].head()



,duration_preAlarm,duration_fault,duration_alarm,duration_isolate
15,NaT,NaT,NaT,2 days 00:36:08.573000
25,NaT,NaT,NaT,7 days 00:02:18.119000
30,NaT,7 days 00:01:00.610000,NaT,NaT
48,NaT,NaT,NaT,7 days 00:02:14.636000
61,NaT,NaT,NaT,7 days 00:02:10.895000


In [57]:
df.head()

,ADDEDAT,COMPANYID,DETECTORID,DURATION,ID,RECORDEDAT,SITEID,SYSTEMGENERATED,WILLSHOWINAPP,device,...,sensor_from_state,descriptor,detector,notification,raw,state,duration_alarm,duration_fault,duration_isolate,duration_preAlarm
0,2024-08-07 17:56:29.842,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},O0uFfYqPcl4PMU3uXxpe,2024-08-07 17:56:28.957,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,initialising rl03 back of house rl03 z1 corrid...,Z109N9L26D172S0,z109n9l26d172s0 - m214 - SC1 retail,initialising rl03 back of house rl03 z1 corrid...,NNNN,NaT,NaT,NaT,NaT
1,2024-08-14 10:49:42.359,n7Tktl6xJA2qDV8RcMpf,yPHI0SRu7cRZ2YNmfDPG,{},xByYZSjb4uJqpLZkLwCa,2024-08-14 10:49:40.278,DO05xnc9uzEwZ1OvRGSP,False,True,191,...,None,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,Z102N9L26D191S0,z102n9l26d191s0 - m214 - SC1 retail,normal rl03 smoke zone 2 rl03 sz2 s3016 ysl,NNNN,NaT,NaT,NaT,NaT
2,2024-09-03 17:05:31.186,n7Tktl6xJA2qDV8RcMpf,hsBtiG1y58ChVue6Mstj,{},EEL0bHavix9GmwdKvd0p,2024-09-03 17:05:16.008,DO05xnc9uzEwZ1OvRGSP,False,True,92,...,None,device missing rl06 smoke zone 1 rl06 sz1 77c ...,Z191N5L13D92S0,z191n5l13d92s0 - m214 - SC1 retail,device missing rl06 smoke zone 1 rl06 sz1 77c ...,??F?,NaT,NaT,NaT,NaT
3,2025-01-09 03:27:26.923,n7Tktl6xJA2qDV8RcMpf,NflPelhGxDx2yF20gn7Y,{},4RVo9ZSGMWFzHhvtUDcd,2025-01-09 03:27:24.867,DO05xnc9uzEwZ1OvRGSP,False,True,172,...,None,normal rl03 back of house rl03 z1 corridor mcp...,Z109N9L26D172S0,z109n9l26d172s0 - SC1 retail - m214,normal rl03 back of house rl03 z1 corridor mcp...,NNNN,NaT,NaT,NaT,NaT
4,2024-08-28 12:18:26.508,n7Tktl6xJA2qDV8RcMpf,5r2EosP2Vnb9cMJsQUA9,{},91S1PYh6HlGxdBqqMf9S,2024-08-28 12:18:25.776,DO05xnc9uzEwZ1OvRGSP,False,True,136,...,None,device missing node 1 ffcp modules eaf-b01-1 s...,Z801N1L3D136S0,z801n1l3d136s0 - m214 - SC1 retail,device missing node 1 ffcp modules eaf-b01-1 s...,??F?,NaT,NaT,NaT,NaT


In [58]:
# Save DataFrame as a CSV file
df.to_csv('Fire_Events_Data_Cleaned.csv', index=False)
